In [34]:
import numpy as np
import cv2
import os

output_dir = "DATASET_dicom_fourier_spectrum"
os.makedirs(output_dir, exist_ok=True)

# for slice in range(218, 300):
# Carregar a imagem em escala de cinza como float64
image = cv2.imread(f'C:\\Users\\CarlosChinen\\OneDrive\\Main\\FEI\\TCC\\Criptografia\\criptografia\\DATASET_dicom_pixel\\pixel_slice_218.png', cv2.IMREAD_GRAYSCALE)
if image is None:
    raise ValueError(f"Imagem não encontrada: slice 218")

image = image.astype(np.float64)  # importante!

# Aplicar a Transformada de Fourier
f = np.fft.fft2(image)
fshift = np.fft.fftshift(f)

# Salvar imagem apenas para visualização (não usada na reconstrução)
magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)  # +1e-8 evita log(0)
spectrum_path = os.path.join(output_dir, f"spectrum_slice_218.png")
cv2.imwrite(spectrum_path, cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))

# Salvar dados da transformada para reconstrução posterior
np.save(os.path.join(output_dir, f"fourier_slice_218.npy"), fshift)

# Opcional: salvar a imagem original como .npy para verificação de hash
np.save(os.path.join(output_dir, f"original_slice_218.npy"), image)


In [35]:
import numpy as np
import os
import cv2

input_dir = "DATASET_dicom_fourier_spectrum"
output_dir = "DATASET_dicom_ifft"
os.makedirs(output_dir, exist_ok=True)

# for slice in range(218, 300):
# Carregar o espectro de Fourier salvo (completo, incluindo fase)
fshift = np.load(os.path.join(input_dir, f"fourier_slice_218.npy"))

# Aplicar ifftshift e transformada inversa
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)

# Extrair apenas a parte real (sem abs, sem uint8)
img_back_real = np.real(img_back)

# Salvar reconstrução como .npy (preservando os dados exatos)
np.save(os.path.join(output_dir, f"reconstructed_slice_218.npy"), img_back_real)

# Salvar imagem apenas para visualização (convertida para 8-bit)
vis_img = cv2.normalize(img_back_real, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
reconstructed_path = os.path.join(output_dir, f"reconstructed_slice_218.png")
cv2.imwrite(reconstructed_path, vis_img)


True

In [39]:
import hashlib

def gerar_hash_sha3_256(dados):
    hash_obj = hashlib.sha3_256(dados)
    return hash_obj.hexdigest()

pasta_original = r"C:\Users\CarlosChinen\OneDrive\Main\FEI\TCC\Criptografia\criptografia\DATASET_dicom_pixel\pixel_slice_218.png"
pasta_descriptografada = r"C:\Users\CarlosChinen\OneDrive\Main\FEI\TCC\Criptografia\criptografia\DATASET_dicom_ifft\reconstructed_slice_218.png"

with open(pasta_original, "rb") as img_file:
    imagem_original = img_file.read()
hash_original = gerar_hash_sha3_256(imagem_original)

with open(pasta_descriptografada, "rb") as img_file:
    imagem_descriptografada = img_file.read()
hash_descriptografada = gerar_hash_sha3_256(imagem_descriptografada)

print(hash_original)
print(hash_descriptografada)

9d4d1ddd03257fa48915299002ba6e4d80f440d936bf4fb975fc76790cb64a7f
cd7a38642e9c70c3811faee617493891e197a4f2aa7b5b45b4930e9f484eb571
